In [1]:
from sympy import *
import numpy as np

In [2]:
L = Symbol('L', positive=True)
D = Symbol('D', positive=True)
x = Symbol('x')
t = Symbol('t')
tau = Symbol('tau')
i, j, imax, jmax = symbols('i, j, imax, jmax', integer=True)

sigma0, sigmaIcorner, cr = symbols('sigma0, sigmaIcorner, c_r ')
Nbar, Nstar, nu_kin = symbols('Nbar, Nstar, nu_kin')

### Making the left-hand side and diffusion terms for all Fourier components

In [3]:
jmax = imax = 2
a = [] # coefficients for Ntot
b = [] # coefficients for NQLL
Ntot = 0
NQLL_LHS = 0
Ntot_LHS = 0
Ntot_RHS_diffusion = 0
NQLL = 0

for j in range(jmax+1):
    
    # Ntot expansion
    a.append(Function('a['+str(j)+']'))
    Ntot_j = a[j](t)*cos(j*pi/L*x)
    Ntot += Ntot_j
    Ntot_LHS += Ntot_j.diff(t)

    # NQLL expansion
    b.append(Function('b['+str(j)+']'))
    NQLL_j = b[j](t)*cos(j*pi/L*x)
    NQLL += NQLL_j
    NQLL_LHS += NQLL_j.diff(t)
    
    # Right-hand side of Ntot equation, diffusion part
    Ntot_RHS_diffusion += D*NQLL_j.diff(x,2)

In [20]:
Ntot

a[0](t) + a[1](t)*cos(pi*x/L) + a[2](t)*cos(2*pi*x/L)

In [4]:
pprint(Ntot_LHS); print('\n')
pprint(NQLL_LHS); print('\n')

   ⎛π⋅x⎞ d                ⎛2⋅π⋅x⎞ d             d          
cos⎜───⎟⋅──(a[1](t)) + cos⎜─────⎟⋅──(a[2](t)) + ──(a[0](t))
   ⎝ L ⎠ dt               ⎝  L  ⎠ dt            dt         


   ⎛π⋅x⎞ d                ⎛2⋅π⋅x⎞ d             d          
cos⎜───⎟⋅──(b[1](t)) + cos⎜─────⎟⋅──(b[2](t)) + ──(b[0](t))
   ⎝ L ⎠ dt               ⎝  L  ⎠ dt            dt         




In [5]:
Ntot_RHS_diffusion

-pi**2*D*b[1](t)*cos(pi*x/L)/L**2 - 4*pi**2*D*b[2](t)*cos(2*pi*x/L)/L**2

### Making the deposition term

In [6]:
sigmaI = sigmaIcorner*(cr*(x/L)**2+(1-cr))
m = (NQLL - (Nbar-Nstar))/(2*Nstar)
sigma_m = sigmaI-m*sigma0
Ntot_RHS_deposition = nu_kin*sigma_m
Ntot_RHS_deposition

nu_kin*(sigmaIcorner*(-c_r + 1 + c_r*x**2/L**2) - sigma0*(-Nbar + Nstar + b[0](t) + b[1](t)*cos(pi*x/L) + b[2](t)*cos(2*pi*x/L))/(2*Nstar))

### Making the ice-QLL equilibration terms

In [7]:
NQLL_eq = Nbar - Nstar*sin(2*pi*Ntot)
NQLL_RHS_equilibration = -(NQLL - NQLL_eq)/tau

### Combining all terms into a single equation

In [8]:
Ntot_RHS = Ntot_RHS_diffusion + Ntot_RHS_deposition
myNtotequation = Eq(Ntot_LHS,Ntot_RHS)
myNtotequation

Eq(cos(pi*x/L)*Derivative(a[1](t), t) + cos(2*pi*x/L)*Derivative(a[2](t), t) + Derivative(a[0](t), t), -pi**2*D*b[1](t)*cos(pi*x/L)/L**2 - 4*pi**2*D*b[2](t)*cos(2*pi*x/L)/L**2 + nu_kin*(sigmaIcorner*(-c_r + 1 + c_r*x**2/L**2) - sigma0*(-Nbar + Nstar + b[0](t) + b[1](t)*cos(pi*x/L) + b[2](t)*cos(2*pi*x/L))/(2*Nstar)))

In [9]:
NQLL_RHS = Ntot_RHS + NQLL_RHS_equilibration
myNQLLequation = Eq(NQLL_LHS,NQLL_RHS)
myNQLLequation

Eq(cos(pi*x/L)*Derivative(b[1](t), t) + cos(2*pi*x/L)*Derivative(b[2](t), t) + Derivative(b[0](t), t), -pi**2*D*b[1](t)*cos(pi*x/L)/L**2 - 4*pi**2*D*b[2](t)*cos(2*pi*x/L)/L**2 + nu_kin*(sigmaIcorner*(-c_r + 1 + c_r*x**2/L**2) - sigma0*(-Nbar + Nstar + b[0](t) + b[1](t)*cos(pi*x/L) + b[2](t)*cos(2*pi*x/L))/(2*Nstar)) + (Nbar - Nstar*sin(pi*(2*a[0](t) + 2*a[1](t)*cos(pi*x/L) + 2*a[2](t)*cos(2*pi*x/L))) - b[0](t) - b[1](t)*cos(pi*x/L) - b[2](t)*cos(2*pi*x/L))/tau)

### Integrating over coordinate space to create a series of equations

In [10]:
print('Working on ',0)
Ntot_LHS0 = Ntot_LHS/(2*L)
Ntot_RHS0 = Ntot_RHS/(2*L)
Ntot_eq0_integrated = integrate(Eq(Ntot_LHS0,Ntot_RHS0),(x, -L, L))
Ntot_LHS_list = [Ntot_LHS0]
Ntot_RHS_list = [Ntot_RHS0]
Ntot_eq_integrated_list = [Ntot_eq0_integrated]
for j in range(1,jmax+1):
    print('Working on ',j)
    LHSj = Ntot_LHS*cos(j*pi*x/L)/L
    RHSj = Ntot_RHS*cos(j*pi*x/L)/L
    Ntot_eq_integrated = integrate(Eq(LHSj,RHSj),(x, -L, L))
    Ntot_eq_integrated_list.append(Ntot_eq_integrated)
print('done')

print('\nLHS ...')
for j in range(jmax+1):
    pprint(Ntot_eq_integrated_list[j].lhs)

print('\nRHS ...')
Ntot_RHS_string_list = []
for j in range(jmax+1):
    Ntot_RHSj_string = str(Ntot_eq_integrated_list[j].rhs)
    Ntot_RHSj_string = Ntot_RHSj_string.replace('(t)','')
    print('for j=', j); pprint(Ntot_RHSj_string); print('\n')

Working on  0
Working on  1
Working on  2
done

LHS ...
d          
──(a[0](t))
dt         
d          
──(a[1](t))
dt         
d          
──(a[2](t))
dt         

RHS ...
for j= 0
-(-L*Nbar*nu_kin*sigma0/(2*Nstar) + 2*L*c_r*nu_kin*sigmaIcorner/3 + L*nu_kin*s
igma0/2 - L*nu_kin*sigmaIcorner + L*nu_kin*sigma0*b[0]/(2*Nstar))/(2*L) + (L*N
bar*nu_kin*sigma0/(2*Nstar) - 2*L*c_r*nu_kin*sigmaIcorner/3 - L*nu_kin*sigma0/
2 + L*nu_kin*sigmaIcorner - L*nu_kin*sigma0*b[0]/(2*Nstar))/(2*L)


for j= 1
(-pi**2*D*b[1]/(2*L) - 2*L*c_r*nu_kin*sigmaIcorner/pi**2 - L*nu_kin*sigma0*b[1
]/(4*Nstar))/L - (pi**2*D*b[1]/(2*L) + 2*L*c_r*nu_kin*sigmaIcorner/pi**2 + L*n
u_kin*sigma0*b[1]/(4*Nstar))/L


for j= 2
(-2*pi**2*D*b[2]/L + L*c_r*nu_kin*sigmaIcorner/(2*pi**2) - L*nu_kin*sigma0*b[2
]/(4*Nstar))/L - (2*pi**2*D*b[2]/L - L*c_r*nu_kin*sigmaIcorner/(2*pi**2) + L*n
u_kin*sigma0*b[2]/(4*Nstar))/L




In [43]:
print('Working on ',0)
nseries = 4
NQLL_LHS0 = NQLL_LHS/(2*L)
NQLL_RHS0 = NQLL_RHS/(2*L)
NQLL_eq0_integrated = integrate(Eq(NQLL_LHS0,NQLL_RHS0.series(x,n=nseries)),(x, -L, L))
NQLL_LHS_list = [NQLL_LHS0]
NQLL_RHS_list = [NQLL_RHS0]
NQLL_eq_integrated_list = [NQLL_eq0_integrated]
for j in range(1,jmax+1):
    print('Working on ',j)
    LHSj = NQLL_LHS*cos(j*pi*x/L)/L
    RHSj = NQLL_RHS*cos(j*pi*x/L)/L
    NQLL_eq_integrated = integrate(Eq(LHSj,RHSj.series(x,n=nseries)),(x, -L, L))
    NQLL_eq_integrated_list.append(NQLL_eq_integrated)
print('done')

print('\nLHS ...')
for j in range(jmax+1):
    pprint(NQLL_eq_integrated_list[j].lhs)

print('\nRHS ...')
NQLL_RHS_string_list = []
for j in range(jmax+1):
    NQLL_RHSj_string = str(NQLL_eq_integrated_list[j].rhs)
    NQLL_RHSj_string = NQLL_RHSj_string.replace('(t)','')
    print('for j=', j); pprint(NQLL_RHSj_string); print('\n')

Working on  0
Working on  1
Working on  2
done

LHS ...
d          
──(b[0](t))
dt         
d          
──(b[1](t))
dt         
d          
──(b[2](t))
dt         

RHS ...
for j= 0
-nu_kin*sigma0*b[2]/(2*Nstar) - nu_kin*sigma0*b[1]/(2*Nstar) - nu_kin*sigma0*b
[0]/(2*Nstar) - b[2]/tau - b[1]/tau - b[0]/tau + nu_kin*sigmaIcorner - nu_kin*
sigma0/2 - c_r*nu_kin*sigmaIcorner - Nstar*sin(2*pi*a[0] + 2*pi*a[1] + 2*pi*a[
2])/tau + Nbar*nu_kin*sigma0/(2*Nstar) + Nbar/tau + 2*L**3*(pi**4*D*b[1]/(4*L*
*5) + 4*pi**4*D*b[2]/L**5 + pi**3*Nstar*a[1]*cos(2*pi*a[0] + 2*pi*a[1] + 2*pi*
a[2])/(2*L**3*tau) + 2*pi**3*Nstar*a[2]*cos(2*pi*a[0] + 2*pi*a[1] + 2*pi*a[2])
/(L**3*tau) + c_r*nu_kin*sigmaIcorner/(2*L**3) + pi**2*b[1]/(4*L**3*tau) + pi*
*2*b[2]/(L**3*tau) + pi**2*nu_kin*sigma0*b[1]/(8*L**3*Nstar) + pi**2*nu_kin*si
gma0*b[2]/(2*L**3*Nstar))/3 - 4*pi**2*D*b[2]/L**2 - pi**2*D*b[1]/L**2 + O(L**5
)


for j= 1
-nu_kin*sigma0*b[2]/Nstar - nu_kin*sigma0*b[1]/Nstar - nu_kin*sigma0*b[0]/Nsta
r - 2*b[2]/tau 

In [ ]:
# LHS1 = LHS*cos(pi*x/L)/L
# RHS1 = RHS*cos(pi*x/L)/L
# eq1_integrated = integrate(Eq(LHS1,RHS1),(x, -L, L))
# eq1_integrated

# LHS2 = LHS*cos(2*pi*x/L)/L
# RHS2 = RHS*cos(2*pi*x/L)/L
# eq2_integrated = integrate(Eq(LHS2,RHS2),(x, -L, L))
# eq2_integrated

# LHS3 = LHS*cos(3*pi*x/L)/L
# RHS3 = RHS*cos(3*pi*x/L)/L
# eq3_integrated = integrate(Eq(LHS3,RHS3),(x, -L, L))
# eq3_integrated